In [15]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from numpy import cos,pi,sin,pi,arccos
from pathlib import Path
from dataclasses import dataclass, field

from datetime import datetime, timezone

from cli import cli

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parse acquisition event log

In [5]:
@dataclass
class PRama_Point:
    x_coord: float
    y_coord: float
    datetime_obj: datetime
    

In [12]:
log_folder_1 = r'\\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage03_SEB'
log_folder_0= r'\\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage02_FZ'
log_folder = r'\\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage01_SEB'

log_list = cli.search(log_folder, '.log')

Searching *.log files in directory:\\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage01_SEB
Scanned filesets are: 

0 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage01_SEB\Acquisition_EVENT_20230507.log
1 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage01_SEB\Acquisition_EVENT_20230508.log
2 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage01_SEB\Acquisition_EVENT_20230509.log
3 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage01_SEB\Acquisition_EVENT_20230510.log
4 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage01_SEB\Acquisition_EVENT_20230511.log
5 \\10.197.125.15\Public\ABP53\abp53_mbes_processing\MBES_EventLogs\stage01_SEB\Acquisition_EVENT_20230512.log


In [13]:
log_picked = cli.pick_from_list(log_list)

Please type two numbers that define the range of filesets you like to copy or move
Please type the first number: 
Please type the second number: 
Do you want to add more elements? (Y/N)


Picked elements:
Acquisition_EVENT_20230507.log
Acquisition_EVENT_20230508.log
Acquisition_EVENT_20230509.log
Acquisition_EVENT_20230510.log
Acquisition_EVENT_20230511.log
Acquisition_EVENT_20230512.log


In [14]:
p_rama_points = []

for log in log_picked:
    with open(log, 'r') as file1:
        file_content = file1.read().splitlines()
        for line in file_content:
            if line.startswith('PR_X'):
                line_content = line.split(',')
                
                datetime_obj = datetime.strptime(line_content[3], 'DT%d.%m.%Y %H:%M:%S.%f')
                datetime_obj.replace(tzinfo=timezone.utc)
                p_rama_point = PRama_Point(x_coord=float(line_content[0][4:]),
                                           y_coord=float(line_content[1][4:]),
                                           datetime_obj=datetime_obj)
                p_rama_points.append(p_rama_point)
                
    output = os.path.join(log_folder, os.path.splitext(os.path.basename(log))[0] + '_P_RAMA.csv')

    with open(output, 'w') as file2:
        file2.write('PR_X,PR_Y,DatetimeUTC\n')
        
        for p_rama_point in p_rama_points:
            file2.write(f'{p_rama_point.x_coord},{p_rama_point.y_coord},{p_rama_point.datetime_obj.strftime("%Y-%m-%dT%H:%M:%S")}\n')